<a href="https://colab.research.google.com/github/saat12d/NaturalLanguageProcessing/blob/main/seq2seq_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Translation

## Data Preparation

In [ ]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

--2023-08-02 06:41:20--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.182.128, 64.233.183.128, 173.194.193.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.182.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  --.-KB/s    in 0.02s   

2023-08-02 06:41:20 (117 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]



Parsing data

In [ ]:
text_file = 'spa-eng/spa.txt'
with open(text_file) as f:
  lines = f.read().split('\n')[:-1]
text_pairs = []

for line in lines:
  english, spanish = line.split('\t')
  spanish = '[start]' + spanish + '[end]'
  text_pairs.append((english, spanish))

import random
print(random.choice(text_pairs))

('I need your advice.', '[start]Necesito tu consejo.[end]')


Creating training, validation, and test sets

In [ ]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples: num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

Vectorizing the English and Spanish text pairs

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import re
import string

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
  lowercase = tf.strings.lower(input_string)
  return tf.strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = 'int',
    output_sequence_length = sequence_length
)

target_vectorization = layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = 'int',
    output_sequence_length = sequence_length + 1,
    standardize = custom_standardization
)

train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

Preparing Datasets for translation task

In [ ]:
batch_size = 64

def format_dataset(eng, spa):
  eng = source_vectorization(eng)
  spa = target_vectorization(spa)
  return ({
      "english": eng,
      # The input Spanish sentence doesn’t include the last token to keep inputs and targets at the same length.
      "spanish": spa[:, :-1]
  }, spa[:, 1:]) # The target Spanish sentence is one step ahead. Both are still the same length (20 words).

def make_dataset(pairs):
  eng_texts, spa_texts = zip(*pairs)
  eng_texts = list(eng_texts)
  spa_texts = list(spa_texts)
  dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
  dataset = dataset.batch(batch_size)
  dataset = dataset.map(format_dataset, num_parallel_calls = 4)
  return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

## Using RNN for seq2seq learning

GRU based Encoder

In [ ]:
embed_dim = 256
latent_dim = 1024

source = keras.Input(shape = (None,), dtype = 'int64', name = 'english')
x = layers.Embedding(vocab_size, embed_dim, mask_zero = True)(source)
encoded_source = layers.Bidirectional(layers.GRU(latent_dim), merge_mode = 'sum')(x)

GRU based decoder and end-to-end model

In [ ]:
past_target = keras.Input(shape = (None,), dtype = 'int64', name = 'spanish')
x = layers.Embedding(vocab_size, embed_dim, mask_zero = True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences = True)
x = decoder_gru(x, initial_state = encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation = 'softmax')(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

Training

In [ ]:
seq2seq_rnn.compile(optimizer = 'rmsprop', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
seq2seq_rnn.fit(train_ds, epochs = 15, validation_data = val_ds)

Epoch 1/15
1302/1302 [==============================] - 146s 98ms/step - loss: 5.4497 - accuracy: 0.2324 - val_loss: 4.5663 - val_accuracy: 0.3044
Epoch 2/15
1302/1302 [==============================] - 106s 81ms/step - loss: 4.4346 - accuracy: 0.3281 - val_loss: 3.8969 - val_accuracy: 0.3851
Epoch 3/15
1302/1302 [==============================] - 106s 81ms/step - loss: 3.8720 - accuracy: 0.3911 - val_loss: 3.4509 - val_accuracy: 0.4437
Epoch 4/15
1302/1302 [==============================] - 106s 81ms/step - loss: 3.4524 - accuracy: 0.4392 - val_loss: 3.1498 - val_accuracy: 0.4837
Epoch 5/15
1302/1302 [==============================] - 106s 81ms/step - loss: 3.1198 - accuracy: 0.4785 - val_loss: 2.9543 - val_accuracy: 0.5126
Epoch 6/15
1302/1302 [==============================] - 106s 81ms/step - loss: 2.8540 - accuracy: 0.5105 - val_loss: 2.7837 - val_accuracy: 0.5376
Epoch 7/15
1302/1302 [==============================] - 106s 81ms/step - loss: 2.6242 - accuracy: 0.5387 - val_loss: 2

Translating new sentences

In [ ]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
  tokenized_input_sentence = source_vectorization([input_sentence])
  decoded_sentence = '[start]'
  for i in range(max_decoded_sentence_length):
    tokenized_target_sentence = target_vectorization([decoded_sentence])
    next_token_predictions = seq2seq_rnn.predict([tokenized_input_sentence, tokenized_target_sentence])
    sampled_token_index = np.argmax(next_token_predictions[0, i, :])
    sampled_token = spa_index_lookup[sampled_token_index]
    decoded_sentence += " " + sampled_token
    if sampled_token == "[end]":
      break
  return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(5):
  input_sentence = random.choice(test_eng_texts)
  print("-")
  print(input_sentence)
  print(decode_sequence(input_sentence))

## Transformer for Machine Translation

Transformer Encoder

In [ ]:
class TransformerEncoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads
    self.attention = layers.MultiHeadAttention(num_heads = num_heads, key_dim = embed_dim)
    self.dense_proj = keras.Sequential([layers.Dense(dense_dim, activation = 'relu'), layers.Dense(embed_dim)])
    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()

  def call(self, inputs, mask = None):
    if mask is not None:
      mask = mask[:, tf.newaxis, :]
    attention_output = self.attention(inputs, inputs, attention_mask = mask)
    proj_input = self.layernorm_1(inputs + attention_output)
    proj_output = self.dense_proj(proj_input)
    return self.layernorm_2(proj_input + proj_output)

  def get_config(self):
    config = super().get_config()
    config.update({
        "embed_dim": self.embed_dim,
        "num_heads": self.num_heads,
        "dense_dim": self.dense_dim
    })
    return config

Transformer Decoder

In [ ]:
class TransformerDecoder(layers.Layer):
  def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
    super().__init__(**kwargs)
    self.embed_dim = embed_dim
    self.dense_dim = dense_dim
    self.num_heads = num_heads
    self.attention_1 = layers.MultiHeadAttention(num_heads = num_heads, key_dim = embed_dim)
    self.attention_2 = layers.MultiHeadAttention(num_heads = num_heads, key_dim = embed_dim)
    self.dense_proj = keras.Sequential([
        layers.Dense(dense_dim, activation = 'relu'),
        layers.Dense(embed_dim)
    ])
    self.layernorm_1 = layers.LayerNormalization()
    self.layernorm_2 = layers.LayerNormalization()
    self.layernorm_3 = layers.LayerNormalization()
    self.supports_masking = True

  def call(self, inputs, encoder_outputs, mask = None):
    causal_mask = self.get_causal_attention_mask(inputs)
    padding_mask = None
    if mask is not None:
      padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
      padding_mask = tf.minimum(padding_mask, causal_mask)
    attention_output_1 = self.attention_1(query = inputs, value = inputs, key = inputs, attention_mask = causal_mask)
    attention_output_1 = self.layernorm_1(inputs + attention_output_1)
    attention_output_2 = self.attention_2(query = attention_output_1, value = encoder_outputs, key = encoder_outputs, attention_mask = padding_mask)
    attention_output_2 = self.layernorm_2(attention_output_1 + attention_output_2)
    proj_output = self.dense_proj(attention_output_2)
    return self.layernorm_3(attention_output_2 + proj_output)

  def get_config(self):
    config = super().get_config()
    config.update({
        "embed_dim": self.embed_dim,
        "num_heads": self.num_heads,
        "dense_dim": self.dense_dim
    })
    return config

  def get_causal_attention_mask(self, inputs):
    input_shape = tf.shape(inputs)
    batch_size, sequence_length = input_shape[0], input_shape[1]
    i = tf.range(sequence_length)[:, tf.newaxis]
    j = tf.range(sequence_length)
    mask = tf.cast(i >= j, dtype = 'int32')
    mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
    mult = tf.concat([tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype = tf.int32)], axis = 0)
    return tf.tile(mask, mult)



Positional Embedding

In [ ]:
class PositionalEmbedding(layers.Layer):

  def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
    super().__init__(**kwargs)
    self.token_embeddings = layers.Embedding(input_dim = input_dim, output_dim = output_dim)
    self.position_embeddings = layers.Embedding(input_dim = sequence_length, output_dim = output_dim)
    self.sequence_length = sequence_length
    self.input_dim = input_dim
    self.output_dim = output_dim

  def call(self, inputs):
    length = tf.shape(inputs)[-1]
    positions = tf.range(start = 0, limit = length, delta = 1)
    embedded_tokens = self.token_embeddings(inputs)
    embedded_positions = self.position_embeddings(positions)
    return embedded_tokens + embedded_positions

  def get_config(self):
    config = super().get_config()
    config.update({
        "output_dim": self.output_dim,
        "sequence_length": self.sequence_length,
        "input_dim": self.input_dim
    })
    return config

End-to-end Transformer model

In [ ]:
embed_dim = 256
dense_dim = 2048
num_heads = 8
vocab_size = 15000
sequence_length = 20

encoder_inputs = keras.Input(shape = (None,), dtype = 'int64', name = 'english')
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape = (None,), dtype = 'int64', name = 'spanish')
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation = 'softmax')(x)

transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

Training the transformer

In [ ]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
1302/1302 [==============================] - 118s 81ms/step - loss: 1.8423 - accuracy: 0.7582 - val_loss: 1.5614 - val_accuracy: 0.7744
Epoch 2/30
1302/1302 [==============================] - 88s 68ms/step - loss: 1.4814 - accuracy: 0.7845 - val_loss: 1.2676 - val_accuracy: 0.7968
Epoch 3/30
1302/1302 [==============================] - 88s 68ms/step - loss: 1.2800 - accuracy: 0.7986 - val_loss: 1.1492 - val_accuracy: 0.8051
Epoch 4/30
1302/1302 [==============================] - 88s 68ms/step - loss: 1.1769 - accuracy: 0.8068 - val_loss: 1.0792 - val_accuracy: 0.8121
Epoch 5/30
1302/1302 [==============================] - 88s 68ms/step - loss: 1.1159 - accuracy: 0.8126 - val_loss: 1.0494 - val_accuracy: 0.8169
Epoch 6/30
1302/1302 [==============================] - 88s 68ms/step - loss: 1.0743 - accuracy: 0.8169 - val_loss: 1.0452 - val_accuracy: 0.8188
Epoch 7/30
1302/1302 [==============================] - 90s 69ms/step - loss: 1.0443 - accuracy: 0.8210 - val_loss: 1.0368 

Translating new sentences with Transformer Model

In [ ]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
  tokenized_input_sentence = source_vectorization([input_sentence])
  decoded_sentence = "[start]"
  for i in range(max_decoded_sentence_length):
    tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]
    predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
    sampled_token_index = np.argmax(predictions[0, i, :])
    sampled_token = spa_index_lookup[sampled_token_index]
    decoded_sentence += " " + sampled_token
    if sampled_token == "[end]":
      break
  return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(5):
  input_sentence = random.choice(test_eng_texts)
  print("-")
  print(input_sentence)
  print(decode_sequence(input_sentence))

-
Can we work on that now?
[start] trabajar eso eso ese ahora[end]               
-
I need the key to decode this message.
[start] la llave de es este               
-
Tom felt cold.
[start] se sintió frío[end]                 
-
It wouldn't surprise me if Tom and Mary got married.
[start] me sorpresa si si tom tom mary se            
-
What do you have against Tom?
[start] qué [UNK] en                 
